# Using MIRAGE to Generate Simulated NIRCam Images

https://github.com/spacetelescope/mirage/blob/master/examples/Imaging_simulator_use_examples.ipynb

Here we use MIRAGE to simulate NIRCam imaging based on HST observations of the galaxy cluster MACS0647+70.  
Our full APT program executes 160 exposures = 4 dithers x 4 filters x 10 detectors.  
Here, we'll just simulate 16 exposures: 4 dithers in the F200W filter in NIRCam Module A (4 detectors).  
After completing this notebook, the next step is to run the JWST pipeline to combine all 16 exposures into one image.

Inputs:  
APT file outputs: .xml, .pointing  
Galaxy catalog incl. RA, Dec, Sersic fit parameters, magnitude

Outputs:  
Simulated NIRCam FITS images (raw, linear)

In [ ]:
import mirage
import os
os.environ["CRDS_DATA"] = "$HOME/crds_cache"
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"

In [ ]:
from os.path import expanduser
home = expanduser("~")

In [ ]:
if 0:  # for users external to STScI
    # Download 343 GB of files (will take some time!)
    from mirage.reference_files import downloader
    download_path = os.path.join(home, 'MIRAGE', 'data')
    os.makedirs(download_path, exist_ok=True)
    downloader.download_reffiles(download_path, instrument='nircam', dark_type='both',  # linearized
                                 skip_darks=False, skip_cosmic_rays=False, skip_psfs=False, skip_grism=True)
    
    #os.environ["MIRAGE_DATA"] = download_path

In [ ]:
os.environ["MIRAGE_DATA"] = "/ifs/jwst/wit/mirage_data"  # internal to STScI

In [ ]:
# For examining outputs
from glob import glob
from scipy.stats import sigmaclip
import numpy as np
from astropy.io import fits

#%matplotlib notebook
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# mirage imports
from mirage import imaging_simulator
from mirage.seed_image import catalog_seed_image
from mirage.dark import dark_prep
from mirage.ramp_generator import obs_generator
from mirage.yaml import yaml_generator

In [ ]:
import yaml

In [ ]:
# Specify the xml and pointing files exported from APT
#xml_file = 'imaging_example_data/example_imaging_program.xml' # all Module B
#pointing_file = 'imaging_example_data/example_imaging_program.pointing'  # 4 dithered exposures
APT_file = os.path.join(home, 'z11', 'APT', 'JWSTz11_NIRCam')
xml_file      = APT_file + '.xml'
pointing_file = APT_file + '.pointing'

In [ ]:
# Source catalogs
target = 'CLG-J0647+7015'  # must correspond to observed target in APT file (?)
cat_dict = {target:{}}
cat_dict[target]['galaxy']       = 'imaging_MACS0647/MACS0647_MIRAGE_galaxy_catalog.cat'
#cat_dict[target]['point_source'] = 'imaging_example_data/ptsrc_catalog.cat'

In [ ]:
reffile_defaults = 'crds'  # Reference file values: crds or crds_full_name
cosmic_rays = {'library': 'SUNMAX', 'scale': 1.0}  # Cosmic ray library and rate
background = 'medium'
pav3 = 12.5  # telescope roll angle
dates = '2022-10-31'  # won't be used by MIRAGE, but will be added to FITS headers

In [ ]:
output_dir     = './imaging_MACS0647/'  # yaml files
simulation_dir = './imaging_MACS0647/'  # simulated images

In [ ]:
datatype = 'linear, raw'  # Save both raw (for JWST pipeline) and linear (processed except for dark current subtraction)

In [ ]:
# 4 pointings x 5 detectors in Module B (as specified in APT)
yfiles = glob(os.path.join(output_dir,'jw*.yaml'))

In [ ]:
# Print info about these files: filter and detector
for yamlfile in np.sort(yfiles):
    with open(yamlfile, 'r') as infile:
        params = yaml.safe_load(infile)
    filt = params['Readout']['filter']
    detector = params['Readout']['array_name'][3:5]
    print(filt, detector, yamlfile)

In [ ]:
params

In [ ]:
# Run the yaml generator
# Skipping because I did this already
if len(yfiles) == 0:
    yam = yaml_generator.SimInput(input_xml=xml_file, pointing_file=pointing_file,
                                  catalogs=cat_dict, cosmic_rays=cosmic_rays,
                                  background=background, roll_angle=pav3,
                                  dates=dates, reffile_defaults=reffile_defaults,
                                  verbose=True, output_dir=output_dir,
                                  simdata_output_dir=simulation_dir,
                                  datatype=datatype)
    yam.create_inputs()

In [ ]:
# 4 filters
# 4 dithers
# 10 detectors
len(yfiles)

In [ ]:
# Select only images in NIRCam Module A
afiles = [yfile for yfile in yfiles if '_nrca' in yfile]
len(afiles)

In [ ]:
# Select only Module A Long Wavelength images: detector A5
a5files = [yfile for yfile in yfiles if 'a5.' in yfile]
a5files

# Create noiseless seed image as a test

In [ ]:
# https://mirage-data-simulator.readthedocs.io/en/latest/seed_images.html
# https://mirage-data-simulator.readthedocs.io/en/latest/quickstart.html
# catalog_seed_image.py

yamlfile = np.sort(a5files)[0]
cat = catalog_seed_image.Catalog_seed()
cat.paramfile = yamlfile
cat.make_seed()

In [ ]:
from astropy.visualization import simple_norm

def show(data, percent=99.6):
    plt.figure(figsize=(12,12))
    norm = simple_norm(data, 'asinh', percent=percent)
    plt.imshow(data,norm=norm)
    plt.colorbar().set_label('DN$^{-}$/s')
    
show(cat.seedimage)

In [ ]:
# Select only Module A images in filter F200W
yamls_to_process = []
for yamlfile in np.sort(afiles):
    with open(yamlfile, 'r') as infile:
        params = yaml.safe_load(infile)
    filt = params['Readout']['filter']
    if filt == 'F200W':
        yamls_to_process.append(yamlfile)
        
len(yamls_to_process)

In [ ]:
yamls_to_process

In [ ]:
# Only create images that haven't been created already
yamls = np.sort(yamls_to_process)
yamls_to_process = []
for yamlfile in yamls:
    outfits = yamlfile.replace('.yaml', '_uncal.fits')
    already_did_it = os.path.exists(outfits)
    havent_done_it_yet = not already_did_it
    if havent_done_it_yet:
        yamls_to_process.append(yamlfile)

In [ ]:
len(yamls_to_process) 

In [ ]:
yamls_to_process = np.sort(yamls_to_process)
yamls_to_process

# Create the simulated images (will take a while)

In [ ]:
for yfile in yamls_to_process:
    print(yfile)
    m = imaging_simulator.ImgSim()
    m.paramfile = yfile
    m.create()

# Show results for one exposure

In [ ]:
raw_image_file = yfile.replace('.yaml', '_uncal.fits')
raw_data = fits.open(raw_image_file)['SCI'].data
print(raw_data.shape)
data = 1. * raw_data[0, 3, :, :] - 1. * raw_data[0, 0, :, :]
show(data)
raw_image_file

In [ ]:
linear_image_file = yfile.replace('.yaml', '_linear.fits')
linear_data = fits.open(linear_image_file)['SCI'].data
show(linear_data[0, 3, :, :])
linear_image_file

The raw data file is now ready to be run through the [JWST calibration pipeline](https://jwst-pipeline.readthedocs.io/en/stable/) from the beginning. If dark current subtraction is not important for you, you can use Mirage's linear output, skip some of the initial steps of the pipeline, and begin by running the [Jump detection](https://jwst-pipeline.readthedocs.io/en/stable/jwst/jump/index.html?highlight=jump) and [ramp fitting](https://jwst-pipeline.readthedocs.io/en/stable/jwst/ramp_fitting/index.html) steps.

---
<a id='mult_sims'></a>
## Simulating Multiple Exposures

Each yaml file will simulate an exposure for a single pointing using a single detector. To simulate multiple exposures, or a single exposure with multiple detectors, multiple calls to the *imaging_simulator* must be made.

### In Series
```python
paramlist = [yaml_a1,yaml_a2,yaml_a3,yaml_a4,yaml_a5]

def many_sim(paramlist):
    '''Function to run many simulations in series
    '''
    for file in paramlist:
        m = imaging_simulator.ImgSim()
        m.paramfile = file
        m.create()
```

### In Parallel

Since each `yaml` simulations does not depend on the others, we can parallelize the process to speed things up:
```python
from multiprocessing import Pool

n_procs = 5 # number of cores available

with Pool(n_procs) as pool:
    pool.map(make_sim, paramlist)
```

In [ ]:
# https://techwiser.com/how-many-cores-does-my-cpu-have/
n_procs = 6 # number of cores available